In [2]:
import json
import pandas as pd
from datetime import datetime as dt
from datetime import datetime, timedelta
import time
import csv
import statsmodels.api as sm
import os
import numpy as np
import math
import requests


In [4]:
df=pd.read_csv('Final_Trades.csv')
df

,pair,time,trader,volume,fee,position_side,ts,year,price
0,wETH,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,0.113902,LONG,1630532698,2021,3752.71
1,wETH,2021-09-10 21:38:09+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,1.078198,SHORT,1631309889,2021,3216.23
2,wETH,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,65.489925,0.065490,LONG,1631361822,2021,3300.40
3,wETH,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,1502.035796,1.502036,LONG,1631413742,2021,3281.75
4,wETH,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,665.329066,0.665329,LONG,1631427248,2021,3321.09
...,...,...,...,...,...,...,...,...,...
31421,wETH,2023-06-30 22:08:12+00:00,0xe013bf246648e8bba68c932c716f99332fcbbff8,323816.563986,323.816564,LONG,1688162892,2023,1935.25
31422,wETH,2023-06-30 22:08:22+00:00,0x1df578bb65ce9f5f4c89cea5c19bbb840f2274e6,226082.076159,300.708881,SHORT,1688162902,2023,1935.25
31423,wETH,2023-06-30 22:12:12+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,109582.110668,109.582111,LONG,1688163132,2023,1934.76
31424,wBTC,2023-06-30 22:12:57+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,81329.547767,81.329548,LONG,1688163177,2023,30507.32


In [5]:
df['pos'] = [1 if x =='LONG' else -1 for x in df['position_side']]
df

,pair,time,trader,volume,fee,position_side,ts,year,price,pos
0,wETH,2021-09-01 21:44:58+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,0.113902,LONG,1630532698,2021,3752.71,1
1,wETH,2021-09-10 21:38:09+00:00,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,113.902144,1.078198,SHORT,1631309889,2021,3216.23,-1
2,wETH,2021-09-11 12:03:42+00:00,0x32ae17db5157cc331993376fb3ef977d28f2f158,65.489925,0.065490,LONG,1631361822,2021,3300.40,1
3,wETH,2021-09-12 02:29:02+00:00,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,1502.035796,1.502036,LONG,1631413742,2021,3281.75,1
4,wETH,2021-09-12 06:14:08+00:00,0xd2776d0d1c161244667d6c58d527b927c5519c5c,665.329066,0.665329,LONG,1631427248,2021,3321.09,1
...,...,...,...,...,...,...,...,...,...,...
31421,wETH,2023-06-30 22:08:12+00:00,0xe013bf246648e8bba68c932c716f99332fcbbff8,323816.563986,323.816564,LONG,1688162892,2023,1935.25,1
31422,wETH,2023-06-30 22:08:22+00:00,0x1df578bb65ce9f5f4c89cea5c19bbb840f2274e6,226082.076159,300.708881,SHORT,1688162902,2023,1935.25,-1
31423,wETH,2023-06-30 22:12:12+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,109582.110668,109.582111,LONG,1688163132,2023,1934.76,1
31424,wBTC,2023-06-30 22:12:57+00:00,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,81329.547767,81.329548,LONG,1688163177,2023,30507.32,1


In [7]:
df['val']=df['price']*df['volume']/(10**7)
df['val'].describe()

count     31426.000000
mean        692.663997
std        4269.372758
min           0.000007
25%           2.820815
50%          15.923584
75%         100.094785
max      125909.284593
Name: val, dtype: float64

In [3]:
traders=df['trader'].unique()
traders

array(['0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4',
       '0x32ae17db5157cc331993376fb3ef977d28f2f158',
       '0x076edc63cd403a0b7986f0647b7d95a0b389cb92',
       '0xd2776d0d1c161244667d6c58d527b927c5519c5c',
       '0x240354769a8e60decf7dd299d2f0d2c96139ae66',
       '0x33311e11c138ff6ca2b8aa0c73d84bbdd6a00773',
       '0x7641670bcb1fed3d5660cc1c372f48e5fc44bdcc',
       '0x9954fb672454d650599b70b10259dfa076d32439',
       '0x932d80023303d002f38eab17adb990af22c8acd1',
       '0x8d7065932db0e04fd73a89d71eaa3df442c3ec90',
       '0x73dbeb057d37c8fcbf5ce0433739f0e8846b2002',
       '0x1a4117a059374530d69e8393dae09d9453c4a383',
       '0xf6e705aba8cd20ceb3ff078c7323943c4d188edd',
       '0xa7f31b620a23656f63fcb0ddcfe185e5d0663b0f',
       '0x6770fb49eb2c28691dde32ffc306829780014d60',
       '0x795058eb53dde3e5ee1328fbaaa79ceded0eb39e',
       '0x97b3307cc17a636b68d071cfea1875079203df2e',
       '0x2e4c0414df90bc7e02a5863202001b82018fb6bf',
       '0xdfc9155536ac305ebdb72c6c70f176723bae

In [13]:
real_pnls=[]
net_qua=[]
tra_pnl=pd.DataFrame()
for t in traders:
    data=df[df['trader']==t]
    bv = data.loc[data['pos'] == 1, 'val'].sum()
    bq = data.loc[data['pos'] == 1, 'volume'].sum()
    sv = data.loc[data['pos'] == -1, 'val'].sum()
    sq = data.loc[data['pos'] == -1, 'volume'].sum()
    bv=round(bv, 2)
    bq=round(bq, 2)
    sv=round(sv, 2)
    sq=round(sq, 2)
    q=min(bq, sq)
    real_pnls.append(sv*q-bv*q)
    net_qua.append(bq-sq)

tra_pnl['trader']=traders
tra_pnl['Realised PnL']=real_pnls
tra_pnl['Net Qty']=net_qua

tra_pnl

,trader,Realised PnL,Net Qty
0,0xff03b9cc5449f8d9ac70df9f88870b9a87a4aeb4,3.796255e+09,2193088.44
1,0x32ae17db5157cc331993376fb3ef977d28f2f158,2.010966e+08,-102835.45
2,0x076edc63cd403a0b7986f0647b7d95a0b389cb92,-7.510631e+08,2813214.80
3,0xd2776d0d1c161244667d6c58d527b927c5519c5c,-3.244864e+11,6191262.10
4,0x240354769a8e60decf7dd299d2f0d2c96139ae66,-8.489368e+10,1664402.78
...,...,...,...
111,0x0b08ec75f91b9bec1e4ba7259f5c08a4dbafdb54,-1.243280e+10,10527367.54
112,0xc3efac8177377dd2517000e9a5afa0ef4c67b200,2.906237e+09,1529949.11
113,0x89f550c35a2c4a4c4abc9c04a5199e86965a8e86,-5.752023e+09,1434830.21
114,0xe0c2e7b4267b1656add89520105faf63e66d4c05,-1.657712e+07,91516.43


In [15]:
tra_pnl['Realised PnL'].sum()

1264174270589632.8

In [16]:

for t in traders:
    data=df[df['trader']==t]
    a=data['pair'].unique()
    if len(a)!=1:
        print(len(a))

4
2
2
3
4
3
2
3
2
3
2
2
3
2
4
4
2
2
2
2
2
2
2
2
2
2
3
2
4
4
3
2
2
2
4
4
2
2
2
2
2
3
4
2
3
3
2
3
2
3
2
4
4
4
2
3
2
4
3
3
2
2
2
2
4
4
3
2
3
3
2
3
2
3
4
4
4
2
2
3
3
3
2
2
2
2
4
3
3
2
4
3
2
2
4
3
2
3
2
2
2
4
2
2
2
2
2
4
3
4
2
